<a href="https://colab.research.google.com/github/maricari/NLP/blob/main/2d_bot_tfidf_spacy_esp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 web

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Bot con Spacy utilizando un corpus de la web


## Alumna: María Carina Roldán

In [ ]:
!pip install spacy-stanza

In [ ]:
import json
import string
import random
import numpy as np

# Para leer y parsear el texto en HTML
import urllib.request
import bs4 as bs

# Para limpiar el texto
import re
import unicodedata

# Para procesar el texto y armar los pipelines
import spacy
import stanza
import spacy_stanza

In [ ]:
# Descarga el diccionario en español y arma el pipeline de NLP
stanza.download("es")
nlp = spacy_stanza.load_pipeline("es")

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### 1. Datos
Se consumirán los datos de una página con refranes alusivos al tiempo.

In [ ]:
raw_html = urllib.request.urlopen('https://www.cervantesvirtual.com/obra-visor/refranes-alusivos-al-tiempo/html/')
raw_html = raw_html.read()

# Parsea el artículo
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Extrae el texto
# Disclaimer!
# Este texto es ad-hoc para esta página. No servirá para una página genérica

article_section = article_html.find('section')
article_text = article_section.text.split('PANIZO RODRIGUEZ, Juliana\n')[1].split('BIBLIOGRAFÍA')[0]
for titulo in ('EL TIEMPO','EL DIA','LOS DIAS DE LA SEMANA','EL SOL','EL CALOR','EL FRIO','LA HELADA','EL AGUA','EL VIENTO','LA PIEDRA, LOS TRUENOS Y LA NIEBLA','LA NOCHE'):
  article_text = article_text.replace(titulo, '')

# Remueve espacios, saltos de línea o tabulación que pudieran haber quedado
text = re.sub(r'\s+', ' ', article_text)


In [ ]:
text

'Para Rodríguez Marín el refrán es "un dicho popular, sentencioso y breve, de verdad comprobada, generalmente simbólico y expuesto en forma poética, que contiene una regla de conducta u otra enseñanza".Los refranes que inserto a continuación alusivos al tiempo, algunos los he recopilado en Valladolid y pueblos de la provincia y otros proceden de las obras señaladas en la bibliografía. La edad de los informantes oscila entre los veintidós y los ochenta y ocho años.Señalan, entre otros, los siguientes aspectos:-El valor del tiempo: El tiempo es oro. El tiempo es la cosa más preciosa del mundo. El tiempo, que es lo que más vale, no lo da Dios de balde. Quien defiende su tiempo, defiende su dinero.-Los efectos curativos del tiempo: El tiempo cura más que el sol. El tiempo es gran médico para el alma y para el cuerpo. El tiempo todo lo cura y todo lo muda. No hay mal que el tiempo no alivie su tormento.-La invitación a aprovechar el tiempo: Aprovecha el tiempo, que vale el cielo. Con el tie

### 2 - Preprocesamiento de datos en español
Elimina tildes, caracteres especiales, números, signos de puntuación

In [ ]:
def preprocess_clean_text(text):    
    # sacar tildes de las palabras:
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # quitar caracteres especiales
    pattern = r'[^a-zA-z0-9.,!?/:;\"\'\s]' # [^ : ningún caracter de todos estos
    # (termina eliminando cualquier caracter distinto de los del regex)
    text = re.sub(pattern, '', text)
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' # igual al anterior pero sin cifras numéricas
    # quitar números
    text = re.sub(pattern, '', text)
    # quitar caracteres de puntuación
    text = ''.join([c for c in text if c not in string.punctuation])
    return text

In [ ]:
sample_text = '-La invitación a aprovechar el tiempo: Aprovecha el tiempo, que vale el cielo.'
preprocess_clean_text(sample_text)


'la invitacion a aprovechar el tiempo aprovecha el tiempo que vale el cielo'

### 3 Pipeline con Spacy

Tokenization → Lemmatization → Remove stopwords → Remove punctuation

In [ ]:
def spacy_process(text):

    doc = nlp(preprocess_clean_text(text))

    # Tokenization & lemmatization
    lemma_list = []
    for token in doc:
        lemma_list.append(token.lemma_)
    # print("Tokenize+Lemmatize:")
    # print(lemma_list)
    
    # Stop words
    filtered_sentence =[]
    for word in lemma_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 

    # Filter punctuation
    filtered_sentence = [w for w in filtered_sentence if w not in string.punctuation]

    # print("Remove stopword & punctuation: ")
    # print(filtered_sentence)
    return filtered_sentence

In [ ]:
spacy_process(sample_text)

['invitacion',
 'aprovechar',
 'tiempo',
 'aprovechar',
 'tiempo',
 'valer',
 'cielo']

### 3 - Arma el corpus

In [ ]:
corpus = text.replace(":",".").split(".") # divide en oraciones
corpus[10:20]

[' El tiempo cura más que el sol',
 ' El tiempo es gran médico para el alma y para el cuerpo',
 ' El tiempo todo lo cura y todo lo muda',
 ' No hay mal que el tiempo no alivie su tormento',
 '-La invitación a aprovechar el tiempo',
 ' Aprovecha el tiempo, que vale el cielo',
 ' Con el tiempo y la paciencia se adquiere la ciencia',
 '-La fugacidad del tiempo',
 ' Vuela el tiempo de corrida, y tras él va nuestra vida',
 ' Tiempo pasado, jamás tornado']

### 5 - Utilizar vectores TF-IDF y la similitud coseno construido con el corpus de la página

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Respuestas random para cuando no hay match
respuestas_sin_match = ["Así es la vida", "No sé qué decirte", "Y bueno", "vos sabrás"]

def generate_response(user_input, corpus_param):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    # la entrada del usuario se usa para tokenizar y vectorizar
    corpus = corpus_param.copy()
    corpus.append(user_input)

    # Crear un vectorizer TFIDF que utilice nuestra funcion "spacy_process"
    # para obtener los tokens lematizados
    word_vectorizer = TfidfVectorizer(tokenizer=spacy_process)

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    vector_matched = similar_vector_values[0][similar_sentence_number]

    if vector_matched == 0: # si la similaridad coseno fue nula (ningún término en común)
        response = respuestas_sin_match[np.random.randint(len(respuestas_sin_match))]
    else:
        response = corpus[similar_sentence_number] # del corpus obtener el documento más similar

    return response

### 6 - Ensayar el sistema
El sistema intentará encontrar la parte de la página que más se relaciona con nuestro texto de entrada. 
La idea es que un humano diga una expresión y el bot responda con una reflexión acorde. Por ejemplo:

* humano: por qué estoy trabajando!
* bot: La noche se ha hecho para descansar y el día para trabajar


In [ ]:
def bot_response(human_text):
    print(f"humano: {human_text}")    
    resp = generate_response(human_text, corpus)
    print(f"bot: {resp}\n")
    return resp

Un ejemplo con match ...

In [ ]:
response = bot_response('Qué tiempo loco!')

humano: Qué tiempo loco!
bot: Tiempo tuviste



Un ejemplo sin match ...

In [ ]:
response = bot_response('estos bots no saben nada de python')

humano: estos bots no saben nada de python
bot: Y bueno



Le pasamos al bot un banco de frases (una a una) para ver qué nos responde 

In [ ]:
frases_humano = ['vacía está mi billetera ...', 'la helada estuvo complicada', 'por qué estoy trabajando!',
                 'el viento se llevó los techos', 'qué cara está la cebolla', 'qué nostalgia!',
                 'Acá pelando papas, y vos?', 'lunes, martes, miercoles, cuándo se termina esta semana...',
                 'hace frío o calor?', 'los pajaritos cantan, la vieja se levanta',
                 'sin gpu', 'se cortó la luz, que macana', 'tormenta mañana???',
                 'se va a inundar todo con la tormenta.', 'Me voy a ver Netflix'  
                 ]
for _, frase in enumerate(frases_humano):
  bot_response(frase)


humano: vacía está mi billetera ...
bot: No sé qué decirte

humano: la helada estuvo complicada
bot:  Indica que después de haber tres heladas llueve

humano: por qué estoy trabajando!
bot: La noche se ha hecho para descansar y el día para trabajar

humano: el viento se llevó los techos
bot: No haciendo viento, no hace mal tiempo

humano: qué cara está la cebolla
bot:  A mal tiempo, buena cara

humano: qué nostalgia!
bot: vos sabrás

humano: Acá pelando papas, y vos?
bot: De entonces acá ya ha llovido algo

humano: lunes, martes, miercoles, cuándo se termina esta semana...
bot: En todas partes tiene cada semana su martes

humano: hace frío o calor?
bot: Calor de mayo, valor da al año

humano: los pajaritos cantan, la vieja se levanta
bot: Ninguno sabe, cuando se levanta, en qué ha de acabar el día

humano: sin gpu
bot: No sé qué decirte

humano: se cortó la luz, que macana
bot: Ya sale mi Juan por su carga de leña; Lunes, sale; martes, llega; miércoles, corta; jueves, seca; viernes, ca

In [ ]:
corpus[-20:]

### El bot es muy lento! :(

El bot tarda muchísimo en dar una respuesta. Inspeccionando el código se pudo determinar que el mayor tiempo es en la tokenización, específicamente en la instanciación `nlp()`, la cual se ejecuta internamente para cada frase del corpus.

In [ ]:
import time

inicio = time.time()
doc = nlp(sample_text)
fin = time.time()
print(f"Instanciando nlp() spacy_stanza: {fin-inicio}")

nlp = spacy.load('en_core_web_sm')

inicio = time.time()
doc = nlp(sample_text)
fin = time.time()
print(f"Instanciando nlp() spacy: {fin-inicio}")


Instanciando nlp() spacy_stanza: 1.0933876037597656
Instanciando nlp() spacy: 0.07859325408935547


El problema es específicamente con la librería `spacy_stanza`. Con la librería `spacy` este fragmento de código es más de 10 veces más rápido.
Por otro lado, el mismo proceso utilizando las librerías de `NLTK` es aún algo más rápido que `spacy`.

(Nota: La última vez que corrí la notebook, Google me denegó la GPU por lo tanto estos tiempos son con CPU)

## Conclusiones
Se construyó un bot que combina los dos bots explicados en clase: Parsear una página de internet pero usando Spacy para poder trabajar con el idioma español.

### Características del bot
- El texto base es una página con refranes. La idea de este bot es que el humano diga una frase y el bot responda con una reflexión o refrán pertinente a la frase.
- Para hacerlo más interesante, cuando no hay match el bot responde una frase aleatoria tomada de una lista prearmada.

### Resultados
- En la mayoría de los casos las respuestas del bot tienen bastante sentido pensando en su objetivo: simplemente responder algo que tenga relación con la frase del humano. Incluso muchas respuestas resultan muy graciosas.
- Se puede ver cómo sirve usar las librerías de español. Por ejemplo a la frase "por qué estoy trabajando!" el bot responde "La noche se ha hecho para descansar y el día para trabajar", es decir que entendió que las palabras trabajando y trabajar tienen el mismo origen. Por otro lado para la frase 'se cortó la luz' la respuesta habla de 'cortar leña'. Si bien semánticamente no hay relación entre las dos oraciones, es claro que para responder, el algoritmo usó la raiz del verbo cortar.

### Mejoras posibles
- El proceso de vectorización y comparación es bastante lento. Se pudo verificar que el problema era con la librería `spacy_stanza`. Una alternativa es usar la librería base `spacy` o la librería `NLTK` pero se perderían las ventajas del idioma español. Otra alternativa de implementación (no interactiva) podría ser armar un banco de preguntas, luego ampliar el corpus con todas las preguntas en un solo paso, y realizar el proceso de preprocesamiento y vectorización una única vez antes de comparar e imprimir todas las preguntas con sus respuestas.
- Se podría guardar el historial de preguntas y la vectorización, para el caso que el humano repita una frase (igual o con las mismas palabras). Además de ahorrar tiempo de procesamiento, el bot podría responder algo diferente (por ejemplo la segunda frase en orden de similitud).
- Se podría poner un threshold para que el bot considere match solo arriba de un valor dado mayor que cero. Por ejemplo a la frase "Acá pelando papas, y vos?" el bot responde "De entonces acá ya ha llovido algo: ..." lo cual no tiene sentido, solo hay una coincidencia en la palabra "acá" (que por otro lado podría haber sido una *stopword*).
- Otra mejora a realizar sería el parseo de la página ya que está hecha ad-hoc para el texto específico que se eligió de ejemplo.
